<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados. 
   Editado por Equipo Docente IIC2233 2018-1 al 2025-1, y extendido con material creado en 2017-2 por Hugo Navarrete e Ignacio Acevedo.</font>
</p>

# Tabla de contenidos

1. [Eventos y señales](#Eventos-y-señales)
    1. [Obtener al emisor de la señal: `sender`](#Obtener-al-emisor-de-la-señal:-sender)
    2. [Eventos de *mouse*](#Eventos-de-mouse)
        1. [*Move Event* permanente](#Move-Event-permanente)
    3. [Eventos del teclado](#Eventos-del-teclado)
    4. [Generar señales personalizadas](#Generar-señales-personalizadas)
    5. [Emisión de eventos con información](#Emisión-de-eventos-con-información)

# Eventos y señales

Recordemos que las interfaces gráficas son aplicaciones construidas con una **arquitectura basada en manejo de eventos**. Esto significa que la aplicación detecta eventos generados por el usuario o por otras partes de la misma aplicación, y los procesa con el elemento apropiado. 

Una aplicación construida con PyQt empieza a detectar eventos una vez que ha entrado en su *main loop*, y esto ocurre luego de llamar al método `exec()` de `QApplication`. La siguiente figura muestra una comparación, mediante diagramas de flujo, entre un programa con una estructura lineal como los que habíamos construido en las semanas anteriores, y un programa con uso de GUI basada en manejo de eventos.

![](img/gui-flowchart.png)

En este modelo existen 3 elementos fundamentales:

- **La fuente del evento**: Corresponde al objeto que genera el cambio de estado o que genera el evento.
- **El objeto evento**: Es el objeto que encapsula el cambio de estado mediante el evento.
- **El objeto destino:** Equivale al objeto al que se le desea notificar del cambio de estado.

Bajo este modelamiento, la **fuente del evento** delega la tarea de manejar el evento al **objeto destino** pasándole el **objeto evento**. Para implementar este modelo, PyQt utiliza un mecanismo de ***signal*** y ***slot*** que le permite manejar los eventos y hacer posible la comunicación entre *widgets*. De esta forma, cuando un evento ocurre, el objeto que es activado emite una señal (*signal*) a la ranura (*slot*) correspondiente, donde un *slot* puede ser cualquier tipo de elemento llamable (*callable*) en Python.

En el siguiente ejemplo, extenderemos el programa anterior que mostraba botones dispuestos en grilla que simulaban un celular, para generar una llamada a la función `boton_clickeado` cada vez que se presione alguno de los botones. Esto se logra conectando el evento que enviará la señal con el *slot* que la recibe. En el caso de los botones, generalmente el método corresponde al evento `clicked`. Mediante el método `connect()` se establece la comunicación entre el evento y el *slot*. Este método recibe una función *llamable* en Python, (es importante no agregar los `()`, ya que con eso le estaríamos entregando el resultado de la función y no el objeto función). El  ejemplo solo muestra la clase `MiVentana`.

Este código se encuentra en el archivo `3-eventos-y-señales/1_manejar-eventos.py`.

```python
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QPushButton, QLabel,
                             QGridLayout, QVBoxLayout)

class MiVentana(QWidget):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.init_gui()

    def init_gui(self) -> None:
        # Agregamos un label para indicar el último botón seleccionado.
        self.label1 = QLabel('Último botón seleccionado:', self)
        self.label2 = QLabel('', self)
        self.grilla = QGridLayout()

        valores = ['1', '2', '3',
                   '4', '5', '6',
                   '7', '8', '9',
                   '*', '0', '#']

        posiciones = [(i, j) for i in range(4) for j in range(3)]

        for i in range(len(valores)):
            posicion = posiciones[i]
            valor = valores[i]
            boton = QPushButton(valor, self)
            """
            Aquí conectamos el evento clicked() de cada botón con el slot
            correspondiente. En este ejemplo, todos los botones usan el
            mismo slot (self.boton_clickeado).
            """
            boton.clicked.connect(self.boton_clickeado)
            self.grilla.addWidget(boton, *posicion)

        # Posicionamos tanto el label como la grilla en un layout vertical.
        vbox = QVBoxLayout()
        vbox.addWidget(self.label1)
        vbox.addWidget(self.label2)
        vbox.addStretch(1)
        vbox.addLayout(self.grilla)
        self.setLayout(vbox)

        self.setGeometry(300, 150, 200, 200)
        self.setWindowTitle('Celular')

    def boton_clickeado(self) -> None:
        """
        Esta función se ejecuta cada vez que uno de los botones de la grilla
        es presionado. Cada vez que el botón genera el evento, la función
        inspecciona cuál botón fue presionado y recupera la posición en que
        utiliza en la grilla.
        """

        # Sender retorna el objeto que fue clickeado.
        # Ahora, la variable boton referencia una instancia de QPushButton.
        boton = self.sender()
        print(boton)

        # Obtenemos el identificador del elemento en la grilla.
        idx = self.grilla.indexOf(boton)

        # Con el identificador obtenemos la posición del ítem en la grilla.
        posicion = self.grilla.getItemPosition(idx)

        # Actualizamos el texto del label2.
        self.label2.setText(f'Botón {idx}, en fila/columna: {posicion[:2]}.')


if __name__ == '__main__':
    def hook(type, value, traceback) -> None:
        print(type)
        print(traceback)

    sys.__excepthook__ = hook
    app = QApplication([])
    form = MiVentana()
    form.show()
    sys.exit(app.exec())
```

**Nota sobre `label.setText`:** En algunos sistemas operativos, la actualización de *labels* con `setText` no se refleja inmediatamente como se esperaría. Si esto ocurre, hay que ejecutar el método `repaint` de `QLabel` para forzar la actualización, como se muestra a continuación:

```python
# label es una instancia de QLabel
label.setText("Nuevo texto")
label.repaint()
```

#### Obtener al emisor de la señal: `sender`

En ocasiones, como en la función `boton_clickeado()` del código anterior, es necesario conocer cuál de los objetos del formulario envió una señal. El método `sender()` retorna el objeto que generó el evento. El siguiente ejemplo utiliza `sender()` para obtener una referencia al objeto que generó el evento y mostrar cuál objeto generó el evento. 

En el mismo ejemplo, puedes notar que se usa la clase `QCoreApplication`, de `QtCore`, la que permite obtener una referencia al objeto `QApplication` que está en ejecución. Esto se utiliza para forzar una salida de la ventana al hacer *click* en el *slot* del tercer botón.

Este código se encuentra en el archivo `3-eventos-y-señales/2_obtener-sender.py`.

```python
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QPushButton, QLabel,
                             QHBoxLayout, QVBoxLayout)
from PyQt5.QtCore import QCoreApplication


class MiVentana(QWidget):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.init_gui()

    def init_gui(self) -> None:
        self.label_estado = QLabel('Estado: -', self)

        """
        El evento de cada botón es conectado con su slot, en este caso,
        el método boton_clickeado(). Importante notar, que al hacer referencia
        al método no se agregar los paréntesis finales.
        """
        self.boton1 = QPushButton('Botón 1', self)
        self.boton1.resize(self.boton1.sizeHint())
        self.boton1.clicked.connect(self.boton_clickeado)

        self.boton2 = QPushButton('Botón 2', self)
        self.boton2.resize(self.boton2.sizeHint())
        self.boton2.clicked.connect(self.boton_clickeado)

        self.boton3 = QPushButton('Salir', self)
        self.boton3.resize(self.boton3.sizeHint())
        self.boton3.clicked.connect(QCoreApplication.instance().quit)

        hbox = QHBoxLayout()
        hbox.addStretch(1)
        hbox.addWidget(self.boton1)
        hbox.addWidget(self.boton2)
        hbox.addWidget(self.boton3)
        hbox.addStretch(1)

        vbox = QVBoxLayout()
        vbox.addStretch(1)
        vbox.addWidget(self.label_estado)
        vbox.addLayout(hbox)
        vbox.addStretch(1)
        self.setLayout(vbox)

        self.setGeometry(200, 100, 300, 200)
        self.setWindowTitle('Sender')

    def boton_clickeado(self) -> None:
        # Esta función registra el objeto que envía la señal del evento
        # y lo refleja mediante el método sender() en label de estado.
        sender = self.sender()
        self.label_estado.setText(f'Estado: Presionado botón "{sender.text()}"')
        self.label_estado.resize(self.label_estado.sizeHint())


if __name__ == '__main__':
    def hook(type, value, traceback) -> None:
        print(type)
        print(traceback)

    sys.__excepthook__ = hook

    app = QApplication([])
    form = MiVentana()
    form.show()
    sys.exit(app.exec())
```


#### Eventos de *mouse*

No solo los botones tienen implementados eventos de interacción con el usuario: todo *widget* tiene distintos gatillos de eventos que se pueden conectar a *slots*. Por ejemplo, la clase `QWidget` incluye los métodos `mousePressEvent` y `mouseReleaseEvent`, que se hacen cargo -el *widget* que lo implementa- del comportamiento del programa al presionar y liberar el *mouse*, respectivamente. Por defecto, estos métodos no hacen algo, pero siempre podemos hacer *override* de él en nuestros propios *widgets*. Similar a como `QPushButton` tenía una palabra reservada para el evento `clicked` para el cual podía vincular un método o función, `mousePressEvent` refleja el evento genérico de un *click* para cualquier *widget*. La diferencia es que en vez de hacer conexión evento-método (`.clicked.connect(...)`), para este tipo de eventos (y los siguientes ejemplos) la conexión ya está definida y es solo necesario sobrescribir el método correspondiente.

Es decir, al sobrescribir los métodos `mousePressEvent` y `mouseReleaseEvent` de un `QWidget` se está definiendo el código que se ejecutará una vez que se haga *click* sobre ese *widget* y cuando se libere el *click* sobre el mismo.

En el siguiente ejemplo, hay un *labels* de color dentro del *widget*. Cuando se haga *click* en el *widget* vamos a verificar donde se hizo *click*, si fue dentro del *label*, y vamos a avisar cuando este es liberado. Además, vamos a guardar un estado que indica si el *click* comenzó presionando sobre el `QLabel` o fuera de este.

Este código se encuentra en el archivo `3-eventos-y-señales/3_mouse-press-y-release-event.py`.

```python
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel
from PyQt5.QtGui import QMouseEvent


class MiVentana(QWidget):
    def __init__(self) -> None:
        super().__init__()
        self.setGeometry(100, 100, 110, 400)
        self.label = QLabel("Haz click en mí", self)
        self.label.setGeometry(10, 10, 90, 100)
        self.label.setStyleSheet("background-color: lightblue;")
        self.click_dentro_del_label = False

    def mousePressEvent(self, event: QMouseEvent) -> None:
        x = event.x()
        y = event.y()
        print(f"El mouse fue presionado en {x},{y}")
        self.click_dentro_del_label = self.label.underMouse()
        if self.click_dentro_del_label:
            print("\tFue presionado dentro del QLabel")
        else:
            print("\tFue presionado fuera del QLabel")

    def mouseReleaseEvent(self, event: QMouseEvent) -> None:
        x = event.x()
        y = event.y()
        print(f"El mouse fue liberado en {x},{y}")

        if self.click_dentro_del_label:
            print("\tAntes se había presionado dentro del QLabel")
        else:
            print("\tAntes habías presionado fuera del QLabel")

    def mouseMoveEvent(self, event: QMouseEvent) -> None:
        x = event.x()
        y = event.y()
        print(f"El mouse se mueve... está en {x},{y}")


if __name__ == "__main__":
    def hook(type, value, traceback) -> None:
        print(type)
        print(traceback)

    sys.__excepthook__ = hook

    app = QApplication([])
    ventana = MiVentana()
    ventana.show()
    sys.exit(app.exec())
```

#### *Move Event* permanente

Otro aspecto en que nuestro programa puede estar pendiente del usuario, es en revisar la posición en la que se encuentra el *mouse* **cuando este está sobre el *widget* (sin necesariamente presionar)**. 

En el siguiente ejemplo, vamos a ver la posición global del *mouse* mediante el evento `mouseMoveEvent`. Para esto, vamos a activar el *tracking* del *mouse* en el `widget`, pero no en `Qlabel`.

Este código se encuentra en el archivo `3-eventos-y-señales/4_mouse-move-event.py`.

```python
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel
from PyQt5.QtGui import QMouseEvent


class MiVentana(QWidget):
    def __init__(self) -> None:
        super().__init__()
        self.setGeometry(100, 100, 110, 400)
        self.label = QLabel("Haz click en mí", self)
        self.label.setGeometry(10, 10, 90, 100)
        self.label.setStyleSheet("background-color: lightblue;")

        """
        Activamos el tracking del mouse en nuestra ventana.
        """
        self.setMouseTracking(True)

        """
        Si además, queremos trackear la posición del mouse en el label,
        se debe descomentar la siguiente línea.
        """
        # self.label.setMouseTracking(True)

    def mouseMoveEvent(self, event: QMouseEvent) -> None:
        x = event.x()
        y = event.y()
        print(f"El mouse se mueve... está en {x},{y}")


if __name__ == "__main__":
    def hook(type, value, traceback) -> None:
        print(type)
        print(traceback)

    sys.__excepthook__ = hook

    app = QApplication([])
    ventana = MiVentana()
    ventana.show()
    sys.exit(app.exec())
```

Existen muchos más eventos relacionados con el *mouse* que puedes revisar explorando la documentación de PyQt en la web. Unos ejemplos son `enterEvent`, `leaveEvent` y `dragEnterEvent`.

## Eventos del teclado

Otra de las maneras más comunes en que una interfaz gráfica genera eventos por parte del usuario es a través del teclado. Cuando un usuario **presiona o libera una tecla** ocurren eventos para los que muchas veces queremos definir un comportamiento. La clase `QWidget` incluye los métodos `keyPressEvent` y `keyReleaseEvent`, que se hace cargo del comportamiento del programa ante la presión y liberar de una tecla, respectivamente. Por defecto, este método no hace nada, pero siempre podemos hacer *override* de el en nuestros propios *widgets*.

Este código se encuentra en el archivo `3-eventos-y-señales/5_key-press-y-release-event.py`.

```python
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel)
from PyQt5.QtGui import QKeyEvent


class MiVentana(QWidget):
    def __init__(self) -> None:
        super().__init__()
        self.inicializa_gui()

    def inicializa_gui(self) -> None:
        self.estado = QLabel('No se está presionando una tecla.', self)
        self.estado.move(20, 10)
        self.resize(self.estado.sizeHint())

        self.setGeometry(300, 300, 290, 150)
        self.setWindowTitle('Teclado')
        self.show()

    def keyPressEvent(self, event: QKeyEvent) -> None:
        """
        Este método maneja el evento que se produce al presionar las teclas.
        """
        self.estado.setText(f'Presionaron la tecla: {event.text()} '
                            f'de código: {event.key()}')
        self.estado.resize(self.estado.sizeHint())

    def keyReleaseEvent(self, event: QKeyEvent) -> None:
        """
        Este método maneja el evento que se produce al liberar una tecla.
        """
        self.estado.setText('No se está presionando una tecla.')
        self.estado.resize(self.estado.sizeHint())


if __name__ == '__main__':
    def hook(type, value, traceback) -> None:
        print(type)
        print(traceback)

    sys.__excepthook__ = hook

    app = QApplication([])
    ventana = MiVentana()
    sys.exit(app.exec())
```

#### Generar señales personalizadas

PyQt permite definir señales personalizadas por el programador, además de los predeterminados sobre el *mouse* y teclado. Para esto se debe crear un objeto que alojará (como atributo) la nueva señal, el que debe ser subclase de `QtCore.QObject`. Dentro de la subclase se crea la nueva señal como una instancia de `QtCore.pyqtSignal`. Finalmente, en los *widgets* involucrados se deben recibir las señales y conectar las funciones que manejan la señal.

- Mediante el método `emit` de `pyqtSignal` se emite la señal (indicando que ocurrió un evento).
- Mediante el método `connect`, también de `pyqtSignal`, se define la función o método a ejecutar cuando la señal es emitida.

El siguiente ejemplo muestra cómo generar una nueva señal entre dos ventanas separadas. La señal se activa al presionar alguna parte de la ventana izquierda, y su efecto es editar el contenido de la ventana derecha.

La señal se crea como un atributo de clase dentro de la clase `VentanaPresionable`, que es quien **emite el evento** cada vez que se presiona la ventana, se **conecta** en el *main* a uno de los métodos de `VentanaQueSeEdita`, y finalmente `VentanaQueSeEdita` es quien **recibe el evento** y edita su contenido cada vez que se emite la señal.

Este código se encuentra en el archivo `3-eventos-y-señales/6_crear-pyqtsignals.py`.

```python
import sys
from PyQt5.QtCore import pyqtSignal
from PyQt5.QtGui import QMouseEvent
from PyQt5.QtWidgets import QApplication, QWidget, QLabel


class VentanaPresionable(QWidget):
    """
    Creamos una señal como atributo de clase.
    """
    senal_escribir = pyqtSignal()

    def __init__(self) -> None:
        super().__init__()
        self.inicializa_gui()

    def inicializa_gui(self) -> None:
        self.etiqueta = QLabel('Presiona esta ventana', self)
        self.etiqueta.move(20, 10)
        self.etiqueta.resize(self.etiqueta.sizeHint())

        self.setGeometry(300, 300, 290, 150)
        self.setWindowTitle('Emite señal')
        self.show()

    def mousePressEvent(self, event: QMouseEvent) -> None:
        """
        Al ejecutar la siguiente línea, se emite la señal,
        y los métodos conectados se llamarán automáticamente.
        """
        self.senal_escribir.emit()


class VentanaQueSeEdita(QWidget):
    def __init__(self) -> None:
        super().__init__()
        self.inicializa_gui()

    def inicializa_gui(self) -> None:
        self.etiqueta = QLabel('', self)
        self.etiqueta.move(20, 10)
        self.etiqueta.resize(self.etiqueta.sizeHint())

        self.setGeometry(700, 300, 290, 150)
        self.setWindowTitle('Recibe señal')
        self.show()

    # Este es el método que conectaremos a la señal
    def edita_etiqueta(self) -> None:
        self.etiqueta.setText('¡Oh! Alguien ha presionado el mouse')
        self.etiqueta.resize(self.etiqueta.sizeHint())


if __name__ == '__main__':
    def hook(type, value, traceback) -> None:
        print(type)
        print(traceback)

    sys.__excepthook__ = hook

    app = QApplication([])
    ventana_click = VentanaPresionable()
    ventana_edit = VentanaQueSeEdita()

    """
    Conectamos la señal con el método que debe activar.
    """
    ventana_click.senal_escribir.connect(ventana_edit.edita_etiqueta)

    sys.exit(app.exec())
```

##### (Abre paréntesis...

1. ¿Por qué se encapsula la instancia `pyqtSignal` dentro de una clase? ¿No se puede instanciar directamente y trabajar con el objeto `pyqtSignal`?
2. ¿Por qué se define como un atributo de clase y no de instancia?
3. ¿Esto último significa que todas las instancias de la clase `VentanaPresionable` compartirán la misma señal?

¡Muy buenas preguntas! Las tres prácticamente se responden de la misma forma: la razón de fondo es la implementación interna de PyQt que requiere estas condiciones. Ahora, específicamente:

1. Las `pyqtSignal` deben estar encapsulada por objetos que hereden de `QObject`. Se pueden instanciar señales fuera de una clase, pero estas no serán capaces de conectarse a funciones o ser emitidas por alguna entidad. ¿Por qué entonces podemos encapsular la señal en `VentanaPresionable`, que no es un `QObject`? Esto es simplemente porque nuestras ventanas heredan de `QWidget` y todo `QWidget` es a su vez un `QObject`.
2. Las instancias de `pyqtSignal` deben ser definidas como atributo de clase y no de instancia por diseño interno de PyQt. Puedes hacer la prueba e instanciarla como un atributo de instancia, y verás que esta no será capaz de conectarse a llamables o de emitirse.
3. Hemos visto que el comportamiento de un atributo de clase es distinto para un atributo de instancia para objetos mutables. En general, si tenemos un atributo de clase mutable (como una lista), si cualquier instancia modifica este atributo, todas las instancias tienen acceso a este atributo y ven el efecto. Entonces, ¿todas las instancias de `MiSenal` compartirían la señal `senal`? Soprendentemente, la respuesta es **no**. Esto se debe a implementación interna de PyQt que logra que a pesar de que se define a nivel de atributo de clase, para cada instancia la señal creada como atributo es única y distinta. Entonces, a pesar de que se defina a ese nivel, podemos estar tranquilos en que cada instancia tendrá su propia señal única.

**Importante:** Al momento de definir el atributo que contiene la señal, este solo puede utilizar caracteres [ASCII](https://es.wikipedia.org/wiki/ASCII), es decir, no se permite el uso de caracteres especiales como la "`ñ`" o el uso de tildes.

##### ...cierra paréntesis)

#### Emisión de eventos con información

En nuestro primer ejemplo básico de señal personalizada, se emitió y recibió un evento que simplemente ocurría y realizaba un efecto. Este no transmitía más información ni datos que describieran el evento que ocurrió. `pyqtSignal` permite una forma para que, al momento de emitir el evento (`emit`), se envíe también información a la función o método conectado y lo reciba como un argumento. Al instanciar la señal `pyqtSignal`, este recibe como argumentos los tipos de objetos que es capaz de enviar mediante `emit` y que recibirán los objetos llamables como argumentos. En el siguiente ejemplo se expande el anterior con tres señales separadas que emiten distinta información.

Este código se encuentra en el archivo `3-eventos-y-señales/7_enviar-informacion-pyqtsignals.py`.

```python
import sys
from PyQt5.QtCore import pyqtSignal
from PyQt5.QtGui import QMouseEvent
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QLineEdit)


class VentanaPresionable(QWidget):
    """
    Creamos las señales como atributos de clase.
    """
    senal_simple = pyqtSignal()                 # Señal simple.
    senal_texto = pyqtSignal(str)               # Señal que permite enviar texto.
    senal_coordenadas = pyqtSignal(int, int)    # Señal que permite enviar dos ints.

    def __init__(self) -> None:
        super().__init__()
        self.inicializa_gui()

    def inicializa_gui(self) -> None:
        self.label = QLabel('Presiona esta ventana', self)
        self.label.move(20, 10)
        self.label.resize(self.label.sizeHint())

        self.etiqueta = QLineEdit(self)
        self.etiqueta.move(20, 60)

        self.setGeometry(300, 300, 290, 150)
        self.setWindowTitle('Emite señal')
        self.setMouseTracking(True)
        self.show()

    def mousePressEvent(self, event: QMouseEvent) -> None:
        """
        Se emite la señal simple, sin argumento.
        """
        self.senal_simple.emit()
        """
        Se emite la señal que permite enviar un str. El contenido de la señal,
        será el texto que contenga la etiqueta de la ventana.
        """
        self.senal_texto.emit(self.etiqueta.text())

    def mouseMoveEvent(self, event: QMouseEvent) -> None:
        """
        Se emite la señal que permite enviar dos ints,
        enviamos la posición del mouse.
        """
        self.senal_coordenadas.emit(event.pos().x(), event.pos().y())


class VentanaQueSeEdita(QWidget):
    def __init__(self) -> None:
        super().__init__()
        self.inicializa_gui()

    def inicializa_gui(self) -> None:
        self.etiqueta_1 = QLabel('', self)
        self.etiqueta_1.move(20, 10)
        self.etiqueta_1.resize(self.etiqueta_1.sizeHint())

        self.etiqueta_2 = QLabel('', self)
        self.etiqueta_2.move(20, 40)
        self.etiqueta_2.resize(self.etiqueta_2.sizeHint())

        self.etiqueta_3 = QLabel('', self)
        self.etiqueta_3.move(20, 70)
        self.etiqueta_3.resize(self.etiqueta_3.sizeHint())

        self.setGeometry(700, 300, 290, 150)
        self.setWindowTitle('Recibe señal')
        self.show()

    def edita_etiqueta_click(self) -> None:
        """
        Este método no tiene argumentos,
        ya que se conectará a una señal simple.
        """
        self.etiqueta_1.setText('¡Oh! Alguien ha presionado el mouse')
        self.etiqueta_1.resize(self.etiqueta_1.sizeHint())

    def edita_etiqueta_texto(self, texto) -> None:
        """
        Este método tiene un argumento,
        el str que se espera del evento conectado.
        """
        self.etiqueta_2.setText(f'Recibí del evento: {texto}')
        self.etiqueta_2.resize(self.etiqueta_2.sizeHint())

    def edita_etiqueta_posicion_mouse(self, x, y) -> None:
        """
        Este método tiene dos argumentos,
        los ints que se espera del evento conectado.
        """
        self.etiqueta_3.setText(f'Recibí posiciones: {x}, {y}')
        self.etiqueta_3.resize(self.etiqueta_3.sizeHint())


if __name__ == '__main__':
    def hook(type, value, traceback) -> None:
        print(type)
        print(traceback)

    sys.__excepthook__ = hook

    app = QApplication([])
    ventana_click = VentanaPresionable()
    ventana_edit = VentanaQueSeEdita()

    """
    Conectamos la señal con el método que cada una debe activar.
    """
    ventana_click.senal_simple.connect(ventana_edit.edita_etiqueta_click)
    ventana_click.senal_texto.connect(ventana_edit.edita_etiqueta_texto)
    ventana_click.senal_coordenadas.connect(ventana_edit.edita_etiqueta_posicion_mouse)

    sys.exit(app.exec())
```

También se pueden enviar estructuras de datos mas complejas como `dict`.